In [ ]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 8.2 MB/s eta 0:00:00


In [ ]:
import nltk
import pandas as pd
import xml.etree.ElementTree as ET
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
import spacy
import numpy as np
from spacy.language import Language
import emoji

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_train = '/content/drive/My Drive/Analyse_des_sentiment/data/train.xml'
file_dev   = '/content/drive/My Drive/Analyse_des_sentiment/data/dev.xml'
file_test  = '/content/drive/My Drive/Analyse_des_sentiment/data/test.xml'

In [ ]:

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def read_xml_to_dataframe(file_path):
    # Analyse du fichier XML
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Création d'une liste vide pour stocker les données
    data = []

    # Parcours des éléments de commentaire
    for comment in root.findall('comment'):
        movie = comment.find('movie').text
        review_id = comment.find('review_id').text
        name = comment.find('name').text
        user_id = comment.find('user_id').text
        note = comment.find('note').text.replace(',', '.')
        commentaire = comment.find('commentaire').text

        # Ajout des données dans la liste sous forme de dictionnaire
        data.append({
            'movie': movie,
            'review_id': review_id,
            'name': name,
            'user_id': user_id,
            'note': note,
            'commentaire': commentaire
        })

    # Création d'un DataFrame
    df = pd.DataFrame(data)
    return df

In [ ]:
#fonction pour le fichier de test
def read_xml_to_dataframeTest(file_path):
    # Analyse du fichier XML
    tree = ET.parse(file_path)
    root = tree.getroot()

    # Création d'une liste vide pour stocker les données
    data = []

    # Parcours des éléments de commentaire
    for comment in root.findall('comment'):
        # Initialisation d'un dictionnaire pour chaque commentaire
        comment_data = {}

        # Extraction de chaque élément si disponible et ajout au dictionnaire
        movie = comment.find('movie')
        if movie is not None:
            comment_data['movie'] = movie.text

        review_id = comment.find('review_id')
        if review_id is not None:
            comment_data['review_id'] = review_id.text

        name = comment.find('name')
        if name is not None:
            comment_data['name'] = name.text

        user_id = comment.find('user_id')
        if user_id is not None:
            comment_data['user_id'] = user_id.text

        note = comment.find('note')
        if note is not None:
            comment_data['note'] = note.text.replace(',', '.')

        commentaire = comment.find('commentaire')
        if commentaire is not None:
            comment_data['commentaire'] = commentaire.text

        # Ajout des données dans la liste sous forme de dictionnaire
        data.append(comment_data)

    # Création d'un DataFrame
    df = pd.DataFrame(data)

    return df


In [ ]:
"""# Récupération des données dans des dataFrame"""

#création DataFrame
data_dev   = read_xml_to_dataframe(file_dev)
data_train = read_xml_to_dataframe(file_train)
data_test = read_xml_to_dataframeTest(file_test)


In [ ]:
data_train

,movie,review_id,name,user_id,note,commentaire
0,229864,review_58145147,AM11,Z20051105131449940367618,4.0,"""Ce n'est pas ce qui se trouve en face de vous..."
1,229864,review_59303833,this is my movies,Z20051220181957240298347,3.5,"(...) En soi, je n'ai rien contre ce genre de ..."
2,229864,review_61916672,Shephard69,Z20121226180301760912584,4.5,"Pour sa troisième réalisation après ""Tron, l'h..."
3,229864,review_1000190786,Tayop,Z20121022182542230468657,3.0,Joseph Kosinski offre de nouvelles images ébou...
4,229864,review_59207755,ned123,Z20061211223247607682175,3.5,J'ai vu un film... qui traite d'un drame humai...
...,...,...,...,...,...,...
665957,245551,review_55820729,Wally L,Z20121109004006553333277,4.0,super realisation et quelques bon acteurs. ce...
665958,245551,review_1001840194,r@ch,Z20100726110927837577369,3.0,Film visuellement très beau et mis en musique ...
665959,245551,review_56242470,Kaiser D.,Z20170721223303710182536,3.5,parti d'une histoire qui parait pour nous bana...
665960,245551,review_58174196,Florent A.,Z20161204004443590736237,3.0,"Je suis mitigé sur ce film.d un coté,j ai plut..."


In [ ]:
!python -m spacy download fr_core_news_sm


2024-01-08 10:25:06.995482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 10:25:06.995544: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 10:25:06.997103: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 10:25:07.006781: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-08 10:25:08.274704: W tensorflow/compiler/tf2

In [ ]:
# Charger le modèle français de spaCy
nlp = spacy.load('fr_core_news_sm')

In [ ]:
from spacy.lang.fr.stop_words import STOP_WORDS

In [ ]:
ponc_words = [" ",".", ",", ";", "'",":", "(", ")", "[", "]", "\"","..", "...", "....", ".....",
                "\xa0", "  ", "   ", "    ", "     ", "      ", "       ", "        ", "…", "…",
                 "*", "–", "/"
                ]

In [ ]:
#récupération des commentaire
message_test = np.array(data_test['commentaire'])
message_dev = np.array(data_dev['commentaire'])
message_train = np.array(data_train['commentaire'])

In [ ]:
pip install --upgrade emoji


In [ ]:
def remplacer_emoticones_par_leur_sens(texte):
    emoticones = {
        ":)": "content",
        ":D": "très content",
        ":(": "triste",
        ":'(": "pleurer",
        ":'-)": "pleurer de joie",
        ":P": "tirer la langue",
        ";)": "clin d'oeil",
        ";P": "clin d'oeil avec langue",
        ":O": "surpris",
        ":/": "perplexe",
        ">:(": "fâché",
        ">:)": "diabolique",
        "<3": "amour",
        "</3": "cœur brisé",
        "XD": "rire aux éclats",
        ":*": "bisou",
        ":-)": "sourire",
        ":-(": "fâché",
        ":-D": "rire",
        ":-P": "faire une grimace",
        ":-|": "neutre",
        ":-/": "sceptique",
        ";-)": "clin d'oeil souriant",
        ":-O": "bouche bée",
        "O:)": "ange",
        ">:O": "crier",
        ":3": "mignon",
        ":-x": "bouche scellée",
        ":^)": "nez souriant",
        ":-]": "sourire carré",
        ":-[": "mécontent",
        ":-c": "très mécontent",
        ":-&": "malade"
    }

    # Replace each emoticon with its segnification
    for emoticone, sens in emoticones.items():
        texte = texte.replace(emoticone, sens)

    return texte


In [ ]:
def replace_emojis_with_meaning(text):
    # This function will replace each emoji in the text with its meaning.
    return emoji.demojize(text, delimiters=("", ""))

In [ ]:
def preprocess_text(text):
    text = str(text)

    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+', '', text)

    # Replace emojis with their meanings
    text = replace_emojis_with_meaning(text)
    text = remplacer_emoticones_par_leur_sens(text)
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Tokenization
    tokens = word_tokenize(text, language="french")

    # Remove stopwords
    tokens = [word for word in tokens if word not in STOP_WORDS]
    print(tokens)
    # Lemmatization with
    doc = nlp(' '.join(tokens))
    lemmas = [token.lemma_ for token in doc]

    return ' '.join(lemmas)

In [ ]:
# Fill NaN values with an empty string
data_train['commentaire'].fillna('', inplace=True)

In [ ]:
data_dev['commentaire'].fillna('', inplace=True)
data_test['commentaire'].fillna('', inplace=True)

In [ ]:
# Apply this function to each comment in my dataset
data_train['processed_commentaire'] = data_train['commentaire'].apply(preprocess_text)
data_train.to_csv('/content/drive/My Drive/processed_data/data_train_t_processed.csv', index=False)
data_dev['processed_commentaire'] = data_dev['commentaire'].apply(preprocess_text)
data_dev.to_csv('/content/drive/My Drive/processed_data/data_dev_t_processed.csv', index=False)

In [1]:
data_test['processed_commentaire'] = data_test['commentaire'].apply(preprocess_text)
data_test.to_csv('/content/drive/My Drive/processed_data/data_test_t_processed.csv', index=False)

NameError: name 'data_test' is not defined